## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#b57edc; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #b57edc">Libraries</p>

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

import re

pd.options.display.max_rows = 1000

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#b57edc; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #b57edc">Intro</p>

도쿄의 **치안** 데이터와 **오시마랜드** 데이터를 활용하여 EDA(탐색적 데이터 분석)를 수행하고자 한다.

 <a href = 'http://www.oshimaland.co.jp'> 오시마랜드(Oshimaland)</a>
>This website showcases buildings with unfortunate past incidents such as suicide cases, corpse disposal incidents, and murder cases.

<a href = 'https://catalog.data.metro.tokyo.lg.jp/dataset?q=%E6%B2%BB%E5%AE%89&sort=score+desc%2C+metadata_modified+desc'>일본 도쿄 공공 데이터</a>

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#b57edc; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #b57edc">EDA</p>

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#068FFF; font-size:80%; text-align:left;padding: 0px; border-bottom: 3px solid #068FFF">Oshima Jieun Data Processing</p>

In [2]:
oshimaJ= pd.read_csv('/Users/genie/Documents/COLLABORATION/AirbnbWise/Oshimaland_data/jieun/oshimaland_dataset_final.csv')
oshimaJ.head()

,info,address,occurred_date,recorded_date
0,819号室で女性死亡,東京都新宿区歌舞伎町一丁目202,平成30年5月23日,平成30年7月4日
1,飛び降り自殺,東京都新宿区歌舞伎町一丁目202アパホテル新宿歌舞伎町タワー↓,令和3年5月18日,令和3年5月26日
2,告知事項あり\r\n※管理会社に直接確認済み\r\n\r\nハイム大成ビル 403号室\r\...,東京都新宿区西新宿七丁目1813,1年以内,令和3年2月23日
3,１４Ｆから女性が飛び降り自殺,東京都新宿区西新宿七丁目1713パレステュディオ新宿WEST,令和元年9月23日,令和元年9月23日
4,事故死,東京都新宿区西新宿七丁目2231,2016,平成28年12月4日


In [3]:
oshimaJ.drop(columns = ['recorded_date'], inplace = True) #* recorded_date(기록 날짜) 칼럼 제거
oshimaJ.rename(columns = {'occurred_date': 'date'}, inplace = True) #* occured_date -> date 칼럼명 변경

In [4]:
#* address 칼럼 가공
#! 일본어 주소는 일반적으로 '시' 또는 '구', '동/읍/면', '번지', 그리고 '건물명' 등의 정보로 구성
#* (1) address의 모든 데이터에 '東京都(도쿄도)' 포함되어 있으므로 제거
oshimaJ['address'] = oshimaJ['address'].str.replace('東京都','')
oshimaJ.tail()

,info,address,date
565,死体発見,武蔵野市西久保三丁目75都築荘101,平成28年2月23日
566,5階 角部屋\r\n告知事項有り,国分寺市戸倉一丁目825 PlaceK,令和4年10月6日
567,6階 角部屋\r\n告知事項有り,国分寺市戸倉一丁目825 PlaceK,令和4年10月6日
568,206号室、506号室\r\n告知事項あり,国分寺市戸倉一丁目825 プレイスK,令和4年7月30日
569,606号室\r\n告知事項あり,国分寺市戸倉一丁目825 プレイスK,令和4年7月30日


In [5]:
#* (2) '시(市) / 구(区) 칼럼(shiku) 추가'
#* oshimaJ['address'].str.contains('市').sum(), 주소에 市(시)가 포함되어 있는 주소는 186개
#* 주소에 市(시)가 없다면 구(区)로 추출
def districtExtract(address):
    if '市' in address:
        extracted_data = address[:address.index('市') + 1]
    elif '市' not in address and '区' in address:
        extracted_data = address[:address.index('区') + 1]
    else: 
        extracted_data = address #* '外国外国板橋二丁目639' 데이터는 시, 구가 모두 없는 유일한 데이터로 삭제해준다.
    return extracted_data

oshimaJ['shiku'] = oshimaJ['address'].apply(districtExtract)

drop_index = oshimaJ[oshimaJ['shiku'] == '外国外国板橋二丁目639'].index
oshimaJ = oshimaJ.drop(drop_index).reset_index(drop=True)
display(oshimaJ.head(), oshimaJ.tail())

,info,address,date,shiku
0,819号室で女性死亡,新宿区歌舞伎町一丁目202,平成30年5月23日,新宿区
1,飛び降り自殺,新宿区歌舞伎町一丁目202アパホテル新宿歌舞伎町タワー↓,令和3年5月18日,新宿区
2,告知事項あり\r\n※管理会社に直接確認済み\r\n\r\nハイム大成ビル 403号室\r\...,新宿区西新宿七丁目1813,1年以内,新宿区
3,１４Ｆから女性が飛び降り自殺,新宿区西新宿七丁目1713パレステュディオ新宿WEST,令和元年9月23日,新宿区
4,事故死,新宿区西新宿七丁目2231,2016,新宿区


,info,address,date,shiku
564,死体発見,武蔵野市西久保三丁目75都築荘101,平成28年2月23日,武蔵野市
565,5階 角部屋\r\n告知事項有り,国分寺市戸倉一丁目825 PlaceK,令和4年10月6日,国分寺市
566,6階 角部屋\r\n告知事項有り,国分寺市戸倉一丁目825 PlaceK,令和4年10月6日,国分寺市
567,206号室、506号室\r\n告知事項あり,国分寺市戸倉一丁目825 プレイスK,令和4年7月30日,国分寺市
568,606号室\r\n告知事項あり,国分寺市戸倉一丁目825 プレイスK,令和4年7月30日,国分寺市


In [6]:
#* (3) town(동/ 읍/ 면) 칼럼 추가 -> town 구역 + 건물 번호 모두 추출
shikuList = list(oshimaJ['shiku'])
def townExtract(address):
    if isinstance(address, str):
        for idx in range(len(shikuList)):
            if shikuList[idx] in address:
                find_data = address[address.find(shikuList[idx]) + len(shikuList[idx]):]
                return find_data

oshimaJ['town'] = oshimaJ['address'].apply(townExtract)
display(oshimaJ.head(), oshimaJ.tail())


,info,address,date,shiku,town
0,819号室で女性死亡,新宿区歌舞伎町一丁目202,平成30年5月23日,新宿区,歌舞伎町一丁目202
1,飛び降り自殺,新宿区歌舞伎町一丁目202アパホテル新宿歌舞伎町タワー↓,令和3年5月18日,新宿区,歌舞伎町一丁目202アパホテル新宿歌舞伎町タワー↓
2,告知事項あり\r\n※管理会社に直接確認済み\r\n\r\nハイム大成ビル 403号室\r\...,新宿区西新宿七丁目1813,1年以内,新宿区,西新宿七丁目1813
3,１４Ｆから女性が飛び降り自殺,新宿区西新宿七丁目1713パレステュディオ新宿WEST,令和元年9月23日,新宿区,西新宿七丁目1713パレステュディオ新宿WEST
4,事故死,新宿区西新宿七丁目2231,2016,新宿区,西新宿七丁目2231


,info,address,date,shiku,town
564,死体発見,武蔵野市西久保三丁目75都築荘101,平成28年2月23日,武蔵野市,西久保三丁目75都築荘101
565,5階 角部屋\r\n告知事項有り,国分寺市戸倉一丁目825 PlaceK,令和4年10月6日,国分寺市,戸倉一丁目825 PlaceK
566,6階 角部屋\r\n告知事項有り,国分寺市戸倉一丁目825 PlaceK,令和4年10月6日,国分寺市,戸倉一丁目825 PlaceK
567,206号室、506号室\r\n告知事項あり,国分寺市戸倉一丁目825 プレイスK,令和4年7月30日,国分寺市,戸倉一丁目825 プレイスK
568,606号室\r\n告知事項あり,国分寺市戸倉一丁目825 プレイスK,令和4年7月30日,国分寺市,戸倉一丁目825 プレイスK


In [7]:
#* oshimaJ['town'].str.contains('丁目').sum(), '丁目' 이 존재하는 데이터는 542개
#* '丁目'이 존재하지 않는 경우, 주소에 숫자가 포함되어 있다면 숫자 이전 문자열 까지 구역으로 추출 하고, 숫자가 없다면 그대로 추출
# def partExtract(town):
#     if '丁目' in town:
#         extracted_data = town[:town.find('丁目')] + '丁目'
#     else:
#         for idx, char in enumerate(town):
#             if any(char.isdigit() for char in town): #* town 주소에서 하나 이상의 숫자가 발견되었는지 확인
#                 match = re.search(r'\d+', town)
#                 extracted_data = town[:match.start()]
#             else:
#                 extracted_data = town[idx]
#     return extracted_data

#oshimaJ['townpart'] = oshimaJ['town'].apply(partExtract)
#! 확인해보니 '大山東町321 フェニックス板橋大山207号室' 이와 같이 숫자가 두번 포함된 경우가 있어 town은 굳이 나누지 않는다.

In [8]:
#* (4) 특수 문자 제거 -> 이 데이터의 경우 '\r', '\n', '↓' 를 제거해야 한다.
def remove_special_characters(text):
    if isinstance(text, str):  #* 문자열인 경우에만 정규표현식 적용
        pattern = r'[\r\n↓]'
        cleaned_text = re.sub(pattern, '', text)
        return cleaned_text
    return text

oshimaJ_cleaned = oshimaJ.applymap(remove_special_characters)

In [9]:
oshimaJ_cleaned.shape

(569, 5)

In [10]:
#* (5) 칼럼 순서 변경
oshimaJ_cleaned = oshimaJ_cleaned.reindex(columns = ['info', 'date', 'address', 'shiku', 'town'])

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#068FFF; font-size:80%; text-align:left;padding: 0px; border-bottom: 3px solid #068FFF">Oshima Yunyoung Data Processing</p>

In [11]:
oshimaY = pd.read_csv('/Users/genie/Documents/COLLABORATION/AirbnbWise/Oshimaland_data/yunyoung/selenium_tokyo_data.csv', index_col = 0)
oshimaY.head()

,district,date,address,content
0,新宿区,平成24年11月5日,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見
1,新宿区,平成24年11月5日,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見
2,新宿区,令和4年,東京都新宿区西新宿一丁目3-9,飛び降り自殺
3,新宿区,NaN,NaN,NaN
4,新宿区,平成20年1月5日,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見


In [12]:
#! 일본어 주소는 일반적으로 '시' 또는 '구', '동/읍/면', '번지', 그리고 '건물명' 등의 정보로 구성
#* (1) address의 모든 데이터에 '東京都(도쿄도)' 포함되어 있으므로 제거
oshimaJ['address'] = oshimaJ['address'].str.replace('東京都','')

In [13]:
oshimaY.rename(columns = {'content': 'info', 'district' : 'shiku'}, inplace = True) #* content -> info 칼럼명 변경
oshimaY = oshimaY.reindex(columns = ['info', 'date','address', 'shiku'])
oshimaY.tail()

,info,date,address,shiku
298,駐車スペース横の部屋で住民男性が首吊り自殺,2005,東京都羽村市緑ヶ丘五丁目1-50,羽村市
299,告知事項,不明,東京都羽村市羽中一丁目3-37,武蔵村山市
300,死体発見,平成24年,東京都武蔵村山市中央一丁目24-3ハウスオブブーケA 1階,昭島市
301,火災による死亡,令和2年9月7日,東京都昭島市田中町一丁目34-9,福生市
302,通路から駐車場へ男性が飛び下り自殺,2016年,東京都福生市志茂156-1,福生市


In [14]:
#* oshimaJ_cleaned 데이터프레임에 맞춰 oshimaY 데이터프레임 변경
display(oshimaJ_cleaned.head(1), oshimaY.tail())

,info,date,address,shiku,town
0,819号室で女性死亡,平成30年5月23日,新宿区歌舞伎町一丁目202,新宿区,歌舞伎町一丁目202


,info,date,address,shiku
298,駐車スペース横の部屋で住民男性が首吊り自殺,2005,東京都羽村市緑ヶ丘五丁目1-50,羽村市
299,告知事項,不明,東京都羽村市羽中一丁目3-37,武蔵村山市
300,死体発見,平成24年,東京都武蔵村山市中央一丁目24-3ハウスオブブーケA 1階,昭島市
301,火災による死亡,令和2年9月7日,東京都昭島市田中町一丁目34-9,福生市
302,通路から駐車場へ男性が飛び下り自殺,2016年,東京都福生市志茂156-1,福生市


In [15]:
#* (2) town 칼럼 추가
shikuList = list(oshimaY['shiku'])
oshimaY['town'] = oshimaY['address'].apply(townExtract)

In [16]:
#* (3) 특수문자 제거 
oshimaY_cleaned = oshimaY.applymap(remove_special_characters)

In [17]:
#* (4) oshimaY 데이터 중 None 값을 NaN으로 대체
oshimaY_cleaned = oshimaY_cleaned.applymap(lambda x: np.nan if x is None else x)

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#068FFF; font-size:80%; text-align:left;padding: 0px; border-bottom: 3px solid #068FFF">Data 탐색</p>

In [18]:
#* 각 데이터 shape
print(oshimaJ_cleaned.shape, oshimaY_cleaned.shape)

(569, 5) (303, 5)


In [19]:
#* 각 데이터 결측치 확인
print(oshimaJ_cleaned.isnull().sum())
print('-' * 15)
print(oshimaY_cleaned.isnull().sum())

info       0
date       3
address    0
shiku      0
town       0
dtype: int64
---------------
info       17
date       20
address    13
shiku       0
town       13
dtype: int64


In [20]:
#* 각 데이터 중복데이터 확인
print(oshimaJ_cleaned.duplicated().sum(), oshimaY_cleaned.duplicated().sum())

4 44


In [21]:
#* 두 데이터 병합
oshima = pd.concat([oshimaJ_cleaned, oshimaY_cleaned]).reset_index(drop = True)
print(oshima.shape)

(872, 5)


In [22]:
#* 중복 데이터 제거
oshima = oshima.drop_duplicates()
oshima = oshima.reset_index(drop=True)
print(oshima.shape)

(824, 5)


In [23]:
oshima.isnull().sum() #! 결측치 처리 고민...

info       10
date       15
address     6
shiku       0
town        6
dtype: int64

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#068FFF; font-size:80%; text-align:left;padding: 0px; border-bottom: 3px solid #068FFF">Tokyo Open Data Processing</p>